In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neural_network import MLPRegressor
from datetime import datetime
import os

# ======================================================================
# مسیر فایل‌ها
# ======================================================================
INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"
OUTPUT_FILE = r"C:\BI\Hybrid_Clustering_Autoencoder_Normal_Modeling.xlsx"
LOG_FILE = r"C:\BI\Hybrid_Model_Log.txt"

os.makedirs(r"C:\BI", exist_ok=True)

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")


log("شروع فرآیند Hybrid Clustering + Autoencoder Normal Modeling")

# ======================================================================
# خواندن داده
# ======================================================================
df = pd.read_excel(INPUT_FILE)
log(f"داده خوانده شد → {len(df):,} ردیف")

# ======================================================================
# ساخت datetime
# ======================================================================
df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"],
                                errors="coerce")
df = df.sort_values("datetime").reset_index(drop=True)

# ======================================================================
# نام‌گذاری خوانا
# ======================================================================
friendly_names = {
    "AssetID_8341": "Lube oil filter diffrential pressure",
    "AssetID_8342": "Lube oil tank level",
    "AssetID_8343": "Lube oil tank temperature",
    "AssetID_8344": "Lube oil tank vapor pressure",
    "AssetID_8346": "Over speed pressure",
    "AssetID_9286": "Main lube oil pump press",  # Target
    "AssetID_9287": "Lube oil line pressure",
}
df.rename(columns=friendly_names, inplace=True)

# حذف ستون‌های غیرمفید
drop_cols = ["id", "RecordDate", "RecordTime"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# ======================================================================
# آماده‌سازی داده‌ها
# ======================================================================
sensor_cols = list(friendly_names.values())
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=sensor_cols)

X = df[sensor_cols]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

log("اسکیل‌سازی انجام شد")

# ======================================================================
# مرحله 1: حذف نویز با DBSCAN
# ======================================================================
db = DBSCAN(eps=0.7, min_samples=50).fit(X_scaled)
labels = db.labels_
mask_normal = labels != -1

normal_data = X_scaled[mask_normal]
log(f"DBSCAN → داده نرمال: {len(normal_data):,} ردیف از {len(X_scaled):,}")

# ======================================================================
# مرحله 2: آموزش Autoencoder
# ======================================================================
input_dim = normal_data.shape[1]

autoencoder = MLPRegressor(
    hidden_layer_sizes=(input_dim // 2, input_dim // 3, input_dim // 2),
    activation="relu",
    solver="adam",
    max_iter=200,
    random_state=42
)

autoencoder.fit(normal_data, normal_data)
log("Autoencoder آموزش دید")

# ======================================================================
# مرحله 3: محاسبه REI
# ======================================================================
reconstructed = autoencoder.predict(X_scaled)
df["REI"] = np.mean((X_scaled - reconstructed)**2, axis=1)

# ======================================================================
# مرحله 4: تشخیص Normal / Anomaly
# ======================================================================
threshold = np.percentile(df["REI"], 95)
df["Status"] = np.where(df["REI"] > threshold, "Anomaly", "Normal")

log(f"Threshold انتخاب شد → {threshold:.6f}")

# ======================================================================
# خروجی آخرین 100 رکورد
# ======================================================================
keep_cols = sensor_cols + ["datetime", "REI", "Status"]
df_out = df[keep_cols].sort_values("datetime").tail(100)

df_out.to_excel(OUTPUT_FILE, index=False)
log(f"فایل خروجی ساخته شد → {OUTPUT_FILE}")

print("\n")
print("="*80)
print("🎯 عملیات با موفقیت انجام شد!")
print("📂 مسیر فایل خروجی:")
print(OUTPUT_FILE)
print("="*80)


شروع فرآیند Hybrid Clustering + Autoencoder Normal Modeling
داده خوانده شد → 10,927 ردیف
اسکیل‌سازی انجام شد
DBSCAN → داده نرمال: 10,503 ردیف از 10,925
Autoencoder آموزش دید
Threshold انتخاب شد → 0.774097
فایل خروجی ساخته شد → C:\BI\Hybrid_Clustering_Autoencoder_Normal_Modeling.xlsx


🎯 عملیات با موفقیت انجام شد!
📂 مسیر فایل خروجی:
C:\BI\Hybrid_Clustering_Autoencoder_Normal_Modeling.xlsx
